In [1]:
import numpy as np
import pandas as pd
import re
from scipy import stats
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
from math import log10
from nltk.tokenize import RegexpTokenizer
import seaborn
seaborn.set()

/home/zwa117/sfuhome/cmpt-353/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
wiki = pd.read_json('wiki-company.json.gz', orient='record', lines=True)
rt = pd.read_json('rotten-tomatoes.json.gz', orient='record', lines=True)
omdb = pd.read_json('omdb-data.json.gz', orient='record', lines=True)
data_temp = wiki.merge(rt, on = 'rotten_tomatoes_id')
data = data_temp.merge(omdb, left_on = 'imdb_id_x', right_on = 'imdb_id')
data

,based_on,cast_member,country_of_origin,director,enwiki_title,filming_location,genre,imdb_id_x,label,made_profit,...,audience_average,audience_percent,audience_ratings,critic_average,critic_percent,imdb_id_y,imdb_id,omdb_awards,omdb_genres,omdb_plot
0,Q2515,"[Q440956, Q312712, Q42869, Q45923, Q4509, Q229...",Q30,[Q2593],Kick-Ass (film),"[Q172, Q661896]","[Q188473, Q1146335, Q157443, Q1535153]",tt1250777,Kick-Ass,1.0,...,3.9,81.0,285168.0,7.0,75.0,tt1250777,tt1250777,19 wins & 61 nominations.,"[Action, Comedy]",Dave Lizewski is an unnoticed high school stud...
1,NaN,"[Q34851, Q151973, Q181887, Q283988, Q294641, Q...",Q145,"[Q51583, Q51472, Q364342]",Cleopatra (1963 film),"[Q387047, Q10400]","[Q645928, Q369747, Q1054574, Q130232, Q1433443]",tt0056937,Cleopatra,0.0,...,3.4,69.0,20475.0,5.9,56.0,tt0056937,tt0056937,Won 4 Oscars. Another 2 wins & 13 nominations.,"[Biography, Drama, History]","In 48 B.C., Caesar pursues Pompey from Pharsal..."
2,Q103411,"[Q42204, Q37876, Q313042, Q200405, Q192912, Q4...",Q30,[Q456008],V for Vendetta (film),"[Q64, Q661896, Q990658, Q756819, Q129143, Q198...","[Q2484376, Q20443008, Q3990883, Q622291, Q7210...",tt0434409,V for Vendetta,1.0,...,3.9,90.0,903220.0,6.8,73.0,tt0434409,tt0434409,7 wins & 27 nominations.,"[Action, Drama, Sci-Fi]",Tells the story of Evey Hammond and her unlike...
3,NaN,"[Q108941, Q81328, Q210447, Q4678990, Q16738674...",Q30,[Q188137],Star Wars: The Force Awakens,"[Q184469, Q189, Q27, Q878, Q23220, Q23229, Q66...","[Q471839, Q319221, Q188473, Q157394]",tt2488496,Star Wars: The Force Awakens,1.0,...,4.3,87.0,227740.0,8.2,93.0,tt2488496,tt2488496,Nominated for 5 Oscars. Another 57 wins & 123 ...,"[Action, Adventure, Fantasy]",30 years after the defeat of Darth Vader and t...
4,NaN,"[Q258064, Q354873, Q506198, Q676094, Q230390, ...",Q30,[Q8877],E.T. the Extra-Terrestrial,[Q65],"[Q471839, Q2143665, Q130232]",tt0083866,E.T. the Extra-Terrestrial,NaN,...,3.5,72.0,32313785.0,9.2,98.0,tt0083866,tt0083866,Won 4 Oscars. Another 47 wins & 34 nominations.,"[Family, Sci-Fi]",After a gentle alien becomes stranded on Earth...
5,NaN,"[Q7516, Q13909, Q461238, Q29086, Q464169, Q229...",Q30,[Q387412],Hackers (film),"[Q60, Q1408, Q661896]","[Q1146335, Q580850, Q19367312, Q959790, Q2484376]",tt0113243,Hackers,NaN,...,3.2,68.0,122197.0,4.7,32.0,tt0113243,tt0113243,N/A,"[Comedy, Crime, Drama]",Dade Murphy was a hacker even as a kid in Seat...
6,Q2009573,"[Q81520, Q191104, Q230055, Q190794, Q28054, Q6...",Q145,[Q282033],GoldenEye,"[Q235, Q2802728, Q142, Q1183, Q21, Q159, Q39, ...","[Q3990883, Q2297927, Q188473]",tt0113189,GoldenEye,NaN,...,3.4,83.0,285261.0,7.0,78.0,tt0113189,tt0113189,Nominated for 2 BAFTA Film Awards. Another 2 w...,"[Action, Adventure, Thriller]",When a deadly satellite weapon system falls in...
7,NaN,"[Q37175, Q228862, Q310944, Q234890, Q978857, Q...",Q142,[Q51552],The Ninth Gate,"[Q90, Q45, Q60, Q29]","[Q1200678, Q590103, Q2484376, Q2973181, Q52162...",tt0142688,The Ninth Gate,1.0,...,3.2,57.0,121437.0,5.0,42.0,tt0142688,tt0142688,1 win & 4 nominations.,"[Mystery, Thriller]","In New York, the money-driven dealer Dean Cors..."
8,Q715768,"[Q24962, Q25014, Q25078, Q166159, Q210741, Q22...",Q145,"[Q25078, Q166159]",Monty Python and the Holy Grail,[Q1252715],"[Q157394, Q319221, Q542475, Q157443, Q622548, ...",tt0071853,Monty Python and the Holy Grail,1.0,...,4.1,95.0,643985.0,8.5,97.0,tt0071853,tt0071853,2 wins & 2 nominations.,"[Adventure, Comedy, Fantasy]","History is turned on its comic head when, in 1..."
9,NaN,"[Q105158, Q255070, Q320093, Q432940, Q236826, ...",Q30,[Q122666],Knight Moves (film),[Q24639],"[Q959790, Q1200678, Q590103, Q2484376]",tt0104627,Knight Moves,NaN,...,2.9,39.0,4114.0,3.7,20.0,tt0104627,tt0104627,1 win.,"[Crime, Drama, Mystery]","A chess grandmaster is in a big tournament, an..."


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9612 entries, 0 to 9611
Data columns (total 30 columns):
based_on              2076 non-null object
cast_member           8583 non-null object
country_of_origin     9586 non-null object
director              8886 non-null object
enwiki_title          9612 non-null object
filming_location      4254 non-null object
genre                 9612 non-null object
imdb_id_x             9612 non-null object
label                 9612 non-null object
made_profit           850 non-null float64
main_subject          2940 non-null object
metacritic_id         5223 non-null object
nbox                  985 non-null float64
ncost                 1071 non-null float64
original_language     9508 non-null object
production_company    5161 non-null object
publication_date      9610 non-null object
rotten_tomatoes_id    9612 non-null object
series                601 non-null object
wikidata_id           9612 non-null object
audience_average      8751 non-nu

In [4]:
def corr_points(x):
    oscar_win = re.search(r'Won (\d+) Oscar', x)
    oscar_nominated = re.search(r'Nominated for (\d+) Oscar', x)
    GoldenG_win = re.search(r'Won (\d+) Golden', x)
    GoldenG_nominated = re.search(r'Nominated for (\d+) Golden', x)
    minor_wins = re.search(r'(\d+) win', x)
    minor_nominations = re.search(r'(\d+) nomination', x)
    score = 0
    if oscar_win:
        score = score + 10 * int(oscar_win.group(1))
    if oscar_nominated:
        score = score + 5 * int(oscar_nominated.group(1))
    if GoldenG_win:
        score = score + 7 * int(GoldenG_win.group(1))
    if GoldenG_nominated:
        score = score + 3 * int(GoldenG_nominated.group(1))
    if minor_wins:
        score = score + int(minor_wins.group(1))
    if minor_nominations:
        score = score + 0.25 * int(minor_nominations.group(1))
    return score

In [5]:
data['weighted_points'] = data['omdb_awards'].map(corr_points)
data = data[['director','cast_member','genre','omdb_awards','audience_percent','weighted_points']]
data = data.dropna(subset = ['director','cast_member','genre','omdb_awards','audience_percent','weighted_points']).reset_index(drop = True)

In [6]:
data

,director,cast_member,genre,omdb_awards,audience_percent,weighted_points
0,[Q2593],"[Q440956, Q312712, Q42869, Q45923, Q4509, Q229...","[Q188473, Q1146335, Q157443, Q1535153]",19 wins & 61 nominations.,81.0,34.25
1,"[Q51583, Q51472, Q364342]","[Q34851, Q151973, Q181887, Q283988, Q294641, Q...","[Q645928, Q369747, Q1054574, Q130232, Q1433443]",Won 4 Oscars. Another 2 wins & 13 nominations.,69.0,45.25
2,[Q456008],"[Q42204, Q37876, Q313042, Q200405, Q192912, Q4...","[Q2484376, Q20443008, Q3990883, Q622291, Q7210...",7 wins & 27 nominations.,90.0,13.75
3,[Q188137],"[Q108941, Q81328, Q210447, Q4678990, Q16738674...","[Q471839, Q319221, Q188473, Q157394]",Nominated for 5 Oscars. Another 57 wins & 123 ...,87.0,112.75
4,[Q8877],"[Q258064, Q354873, Q506198, Q676094, Q230390, ...","[Q471839, Q2143665, Q130232]",Won 4 Oscars. Another 47 wins & 34 nominations.,72.0,95.50
5,[Q387412],"[Q7516, Q13909, Q461238, Q29086, Q464169, Q229...","[Q1146335, Q580850, Q19367312, Q959790, Q2484376]",N/A,68.0,0.00
6,[Q282033],"[Q81520, Q191104, Q230055, Q190794, Q28054, Q6...","[Q3990883, Q2297927, Q188473]",Nominated for 2 BAFTA Film Awards. Another 2 w...,83.0,3.50
7,[Q51552],"[Q37175, Q228862, Q310944, Q234890, Q978857, Q...","[Q1200678, Q590103, Q2484376, Q2973181, Q52162...",1 win & 4 nominations.,57.0,2.00
8,"[Q25078, Q166159]","[Q24962, Q25014, Q25078, Q166159, Q210741, Q22...","[Q157394, Q319221, Q542475, Q157443, Q622548, ...",2 wins & 2 nominations.,95.0,2.50
9,[Q122666],"[Q105158, Q255070, Q320093, Q432940, Q236826, ...","[Q959790, Q1200678, Q590103, Q2484376]",1 win.,39.0,1.00


In [7]:
def extract_first_2(x):
    if len(x) <= 2:
        return x
    else:
        return x[:2]
    
def extract_first_3(x):
    if len(x) <= 3:
        return x
    else:
        return x[:3]

def extract_first_4(x):
    if len(x) <= 4:
        return x
    else:
        return x[:4]

# This does not work. Probably because it is not the latest Pandas
# temp = data['director'].apply(extract_first_2, args = (1,))
data['director'] = data['director'].apply(extract_first_2)
data['cast_member'] = data['cast_member'].apply(extract_first_4)
data['genre'] = data['genre'].apply(extract_first_4)

In [8]:

def to_number(x):
    #transfer Non-numeric to number array
    list = []
    size = 0
    num = []
    num1 = []
    x_size = x.size
    for i in range(x_size):
        for j in range(len(x[i])):
            if x[i][j] not in list:
                list.append(x[i][j])
                size = size + 1
                temp = [0]*x_size
                temp[i] = 1
                num.extend(temp)
            else:
                num[(list.index(x[i][j]))*(x_size)+i] = 1
    for i in range(x_size):
        for j in range(size):
            num1.append(num[i+j*x_size])
    array = np.asarray(num1).reshape((x_size,size))
    print('to number array done')
    print('array size ', size)
    print('sample size ', x_size)
    return array

def array_to_list(x):
    # transfer array to list
    list = [[] for i in range(x.shape[0])]
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            list[i].append((x[i][j]))
    return list

def small_feature_index(x,y,z):
    #delete elements that shows no more than z times
    x_size = x[y].size
    category = []
    times = []
    x1 = []
    y1 =[]
    position = [[] for i in range(x_size)]
    for i in range(x_size):
        for j in range(len(x[y][i])):
                if x[y][i][j] not in category:
                    category.append(x[y][i][j])
                    times.append(1)
                    x1.append([i])
                    y1.append([j])
                else:
                    index = category.index(x[y][i][j])
                    times[index] = times[index] + 1 
                    x1[index].append(i)
                    y1[index].append(j)
    for i in range(len(times)):
        if times[i] <= z:
            for j in range(times[i]):
                position[x1[i][j]].append(y1[i][j])
    for i in range(x_size):
        if len(position[i]) != 0:
            for j in sorted(position[i], reverse=True):
                del x[y][i][j] 
    for i in range(x_size):
        if len(x[y][i]) == 0:
            x = x.drop(i)
    x = x.reset_index(drop = True)
    print('delete', y, 'rare parameter done')
    return x

In [9]:
def combine_list(x,y):
    #combine two 2d lists together
    for i in range(len(x)):
        x[i].extend(y[i])
    return x

def model_test(model,X_test,y_test,a):
    # Using model "model" and input data "X_test", "y_test"
    accept = 0
    real = np.asarray(y_test, dtype="float64")
    predict = np.asarray(model.predict(X_test), dtype="float64")
    difference = np.absolute(real - predict)
    for i in range(real.size):
        if difference[i] <= a:
            accept = accept + 1
    score = accept/difference.size
    print ('The test score is: ', score)
    return 0
len(data)

7510

In [10]:
data = small_feature_index(data, 'cast_member', 1)
len(data)

delete cast_member rare parameter done


6858

In [11]:
data = small_feature_index(data, 'genre', 1)
len(data)

delete genre rare parameter done


6856

In [12]:
data = small_feature_index(data, 'director', 1)
len(data)

delete director rare parameter done


5000

In [13]:
def transfer_to_number(x): 
    types =[]
    type_size = 0
    for i in range(len(x)):
        for j in x[i]:
            if j not in types:
                types.append(j)
                type_size = type_size + 1
                
    two_D_array = [x[:] for x in [[0] * len(types)] * len(x)] 
    
    for i in range(len(x)):
        for j in x[i]:
            index = 0
            while j != types[index]:
                index = index + 1
            two_D_array[i][index] = 1        
    
    #print(len(two_D_array))
    print('There are ', len(two_D_array[0]), ' different types in total.')
    print('There are ', len(two_D_array), ' records collected.')
    #print(len(two_D_array[0]))
    # print(types)
    # print(type_size)
    return two_D_array

In [14]:
director = transfer_to_number(data['director'])

There are  1366  different types in total.
There are  5000  records collected.


In [15]:
cast_member= transfer_to_number(data['cast_member'])

There are  4099  different types in total.
There are  5000  records collected.


In [16]:
genre = transfer_to_number(data['genre'])

There are  151  different types in total.
There are  5000  records collected.


In [17]:
X_2 = combine_list(director,cast_member)

In [18]:
X_2 = combine_list(X_2, genre)

In [19]:
y_nominations = np.asarray(data['weighted_points'], dtype="|S6")
y_ratings = np.asarray(data['audience_percent'], dtype="|S6")

In [20]:
print(y_nominations)

[b'34.25' b'45.25' b'13.75' ... b'0.0' b'7.25' b'1.5']


In [21]:
print(y_ratings)

[b'81.0' b'69.0' b'90.0' ... b'79.0' b'78.0' b'33.0']


In [22]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_nominations)

In [23]:
model_2 = MLPClassifier(solver='lbfgs', hidden_layer_sizes=(25,15,5))

In [24]:
model_2.fit(X_train_2, y_train_2)
model_2.predict(X_test_2)
model_test(model_2, X_test_2,y_test_2,4)


The test score is:  0.5416


0

In [25]:
model_2.score(X_train_2, y_train_2)

0.39253333333333335